In [1]:
import random
import numpy as np
import math
import matplotlib.pyplot as plt
import time

"""
This is a refactored version of the EFX allocation finder using Simulated Annealing.
Key optimizations:
1.  Uses a list of sets for `bundles` for efficient item transfers.
2.  Maintains a `agent_utilities` cache for O(1) utility lookups of an agent's own bundle.
3.  The main loop performs an efficient delta-update of the potential function, reducing
    the complexity of each step from O(n^2 * m) to O(n * m).
"""

# --- Core Helper Functions ---

def _count_violations_for_pair(envious_idx, envied_idx, v, bundles, agent_utilities):
    """Counts EFX violations for a single ordered pair (i,j) using the utility cache."""
    # Get the envious agent's utility for their own bundle from the cache (O(1) lookup)
    u_envious_own = agent_utilities[envious_idx]

    # Compute the utility for the other bundle on the fly
    envied_bundle = bundles[envied_idx]
    u_envious_of_envied = sum(v[envious_idx][item] for item in envied_bundle)

    violations = 0
    for item_k in envied_bundle:
        if u_envious_of_envied - v[envious_idx][item_k] > u_envious_own:
            violations += 1
    return violations

def calculate_full_potential(n, v, bundles, agent_utilities):
    """Calculates the total potential from scratch. Used only for initialization."""
    total_violations = 0
    for i in range(n):
        for j in range(n):
            if i == j: continue
            total_violations += _count_violations_for_pair(i, j, v, bundles, agent_utilities)
    return total_violations

# --- Initialization Functions ---

def initialize_allocation(n, m, v):
    """
    Creates a random allocation and all necessary cached data structures.
    Returns:
        bundles (list of sets): bundles[i] is the set of items for agent i.
        owner (list): owner[j] is the agent who owns item j.
        agent_utilities (list): agent_utilities[i] = u_i(A_i).
    """
    bundles = [set() for _ in range(n)]
    owner = [-1] * m
    agent_utilities = [0.0] * n

    for item_j in range(m):
        agent_i = random.randint(0, n - 1)
        bundles[agent_i].add(item_j)
        owner[item_j] = agent_i
        agent_utilities[agent_i] += v[agent_i][item_j]

    return bundles, owner, agent_utilities

# --- Potentially better initialization: start with the welfare maximizing allocation ---
def initialize_allocation_from_optimal_welfare(n, m, v):
    """
    Creates an allocation that maximizes social welfare and all necessary cached data structures.
    Returns:
        bundles (list of sets): bundles[i] is the set of items for agent i.
        owner (list): owner[j] is the agent who owns item j.
        agent_utilities (list): agent_utilities[i] = u_i(A_i).
    """
    bundles = [set() for _ in range(n)]
    owner = [-1] * m
    agent_utilities = [0.0] * n

    for item_j in range(m):
        agent_i = -1 # will give item_j to the agent that likes it the most
        highest_value_for_j = -1
        for i in range(n):
            if v[i][item_j] > highest_value_for_j:
                agent_i = i
                highest_value_for_j = v[i][item_j]

        bundles[agent_i].add(item_j)
        owner[item_j] = agent_i
        agent_utilities[agent_i] += v[agent_i][item_j]

    return bundles, owner, agent_utilities

def generate_random_valuations_int(n, m):
    """Generates the valuation matrix using random values."""
    return [[random.uniform(1, 1000) for _ in range(m)] for _ in range(n)]

def generate_random_valuations(n, m):
    """Generates the valuation matrix using random values."""
    return [[random.uniform(0, 1) for _ in range(m)] for _ in range(n)]

def sample_correlated_valuations(n, m, correlation_strength=0.8, seed=None):
    """Samples additive valuations for agents that are correlated."""
    if seed is not None:
        np.random.seed(seed)
    assert 0 <= correlation_strength <= 1, "correlation_strength must be between 0 and 1"
    shared = np.random.uniform(0, 1, size=m)
    v = [[0 for _ in range(m)] for _ in range(n)]
    for i in range(n):
        noise = np.random.uniform(0, 1, size=m)
        for j in range(m):
            v[i][j] = correlation_strength * shared[j] + (1 - correlation_strength) * noise[j]
    return v

# --- Functions to save the last instance (valuation and initial/final allocation) to a file in case assert breaks ---
# Save the valuation matrix to a file
def write_valuation_to_file(n, m, matrix):
    with open('valuation_matrix.txt', 'w') as f:
        for row in matrix:
            # Convert each element to string and join with a separator
            row_str = ' '.join(map(str, row))
            f.write(row_str + '\n')
    return True

# Save the allocation matrix to a file
def write_allocation_to_file(n, m, bundles, filename):
    # First we convert the allocation to matrix format
    matrix = [[0 for e in range(m)] for e in range(n)]
    for i in range(n):
        for j in bundles[i]:
            matrix[i][j] = 1

    with open(filename, 'w') as f:
        for row in matrix:
            # Convert each element to string and join with a separator
            row_str = ' '.join(map(str, row))
            f.write(row_str + '\n')
    return True

# --- Main Simulation ---

def find_efx_allocation(n, m):
    # 1. SETUP
    v = generate_random_valuations(n, m)

    # Replace with the next line if correlated valuations are desired instead
    #v = sample_correlated_valuations(n, m, correlation_strength=0.9) #, seed=42)

    print(f"Generated instance of size {n} agents X {m} items")
    print("The valuation matrix is::", v)

    # write_valuation_to_file(n, m, v)

    num_steps = 0 # Count the total number of steps until reaching an EFX allocation

    num_restarts = 0

    while True: # Loop for restarts
        num_restarts += 1

        # 2. INITIALIZATION
        bundles, owner, agent_utilities = initialize_allocation(n, m, v)

        # Save the initial allocation to a file
        # write_allocation_to_file(n, m, bundles, 'allocation_initial_matrix.txt')

        f = calculate_full_potential(n, v, bundles, agent_utilities)

        best_f_value = f
        T = 5.0  # Initial temperature
        T_min = 0.0001
        cooling_rate = 0.99

        #print(f"\n--- Restart #{num_restarts} ---")
        print(f"Initial potential: {f}")

        # 3. SIMULATED ANNEALING LOOP
        while T > T_min and best_f_value > 0:
            steps_per_temp = 100 * n * m # 10 * int(m * n / T) # number of steps at the current temperature T

            for _ in range(steps_per_temp):
                if best_f_value == 0: break

                # a) Propose a random move
                item_to_move = random.randint(0, m - 1)
                owner_old = owner[item_to_move]
                owner_new = random.randint(0, n - 1)
                while owner_new == owner_old:
                    owner_new = random.randint(0, n - 1)

                # b) Calculate the change in potential (the delta)
                affected_agents = {owner_old, owner_new}

                # Calculate the part of the potential related to these agents BEFORE the move
                old_slice_f = 0
                for i in range(n):
                    for j in affected_agents:
                        if i != j: old_slice_f += _count_violations_for_pair(i, j, v, bundles, agent_utilities)
                for i in affected_agents:
                    for j in range(n):
                        if i != j and j not in affected_agents:
                            old_slice_f += _count_violations_for_pair(i, j, v, bundles, agent_utilities)

                # c) Tentatively apply the move to the data structures
                bundles[owner_old].remove(item_to_move)
                bundles[owner_new].add(item_to_move)
                agent_utilities[owner_old] -= v[owner_old][item_to_move]
                agent_utilities[owner_new] += v[owner_new][item_to_move]

                # d) Calculate the part of the potential AFTER the move
                new_slice_f = 0
                for i in range(n):
                    for j in affected_agents:
                        if i != j: new_slice_f += _count_violations_for_pair(i, j, v, bundles, agent_utilities)
                for i in affected_agents:
                    for j in range(n):
                        if i != j and j not in affected_agents:
                            new_slice_f += _count_violations_for_pair(i, j, v, bundles, agent_utilities)

                # e) Decide whether to accept the move
                delta_f = new_slice_f - old_slice_f

                accept_move = False
                if delta_f < 0:
                    accept_move = True
                else:
                    if T > 0:
                        acceptance_probability = math.exp(-delta_f / T)
                        if random.uniform(0, 1) < acceptance_probability:
                            accept_move = True

                # f) Finalize or revert the move
                if accept_move:
                    owner[item_to_move] = owner_new
                    f += delta_f
                    #print("Moving to an allocation with function value", f)

                else:
                    # Revert all changes
                    bundles[owner_new].remove(item_to_move)
                    bundles[owner_old].add(item_to_move)
                    agent_utilities[owner_new] -= v[owner_new][item_to_move]
                    agent_utilities[owner_old] += v[owner_old][item_to_move]

                if f < best_f_value:
                    best_f_value = f

                num_steps += 1

            if best_f_value == 0: break # Found EFX allocation
            T *= cooling_rate

        print(f"End of SA run. Best potential found: {best_f_value}. Total steps: {num_steps}")
        if best_f_value == 0:
            print("\n>>> EFX allocation found! <<<")
            break

    # Save the final allocation to file for cross-check
    #write_allocation_to_file(n, m, bundles, 'allocation_final_matrix.txt')

    assert best_f_value == 0, "The potential should be zero at the end"

    return num_steps # for accounting purposes

    #print(f"\nTotal restarts required: {num_restarts}")


    #print(f"\nTotal restarts required: {num_restarts}")
if __name__ == '__main__':

    # --- Initialization Functions ---
    n = 4
    m = 25

    start_time = time.time()

    # The next line will generate a random valuation matrix and run the algorithm for it
    # If desired to start with a specific valuation matrix, change the line where the matrix v is initialized in find_efx_allocation(n,m)

    num_steps = find_efx_allocation(n, m)

    end_time = time.time()

    total_time = end_time - start_time

    print("Finished experiment in " + str(total_time) + "seconds\n")

    print("The number of steps is:", num_steps)


Generated instance of size 4 agents X 25 items
The valuation matrix is:: [[0.5310502005168026, 0.4256875168755122, 0.2701453927778916, 0.4901096485574926, 0.20402676973458755, 0.022624155704694116, 0.15725617817389048, 0.043617970758856206, 0.5218118220306571, 0.1373323032574828, 0.7378357278798469, 0.7990500311583163, 0.3942143954845245, 0.09653445141159334, 0.5353534909156318, 0.17851500629858164, 0.75226924267157, 0.07236416337490881, 0.6611103572110842, 0.24391358300914834, 0.08789427039147413, 0.963749636923117, 0.15348027698055478, 0.2857142052520406, 0.32890084357305027], [0.24576757389678028, 0.02520755508324468, 0.5013133703912771, 0.6708992935197593, 0.590521801259175, 0.775817118695217, 0.4949002734190022, 0.7991037931023397, 0.19631509705237338, 0.7190196175440285, 0.8198962637092228, 0.7158908932832982, 0.5149581851083557, 0.8506156807092675, 0.8070500192031169, 0.16788922823298946, 0.7188367405202414, 0.7592415692687605, 0.8942516471742095, 0.2824746148952604, 0.097072423